In [17]:
from pathlib import Path
import rtoml
config_path = Path("/Users/yiqin/Documents/PythonProjects/GraspDataProcessing-script/scripts/config.toml")

In [18]:
from types import SimpleNamespace
from pathlib import Path
from typing import Dict, Tuple, List, Optional, Union
def load_config(
                    config_path: Union[str, Path]
                    ):
    """加载TOML配置文件并进行类型转换和数据处理"""
    # 转换为Path对象
    config_path = Path(config_path)
    
    # 使用 rtoml 读取TOML文件
    config = rtoml.load(config_path)

    
    return SimpleNamespace(**config)

In [19]:
test = load_config(config_path)

In [24]:
test.ml_config

{'descriptors_with_subshell_info': True,
 'high_prob_percentile': 95,
 'overfitting_threshold': 0.1,
 'underfitting_threshold': -0.05,
 'include_wrong_level_negatives': True}

In [7]:
dddd = 3e-9

In [2]:
number = 1e-9
exponent = int(f"{number:.0e}".split('e')[-1])

In [3]:
exponent

-9

In [8]:
f"{dddd:.0e}"

'3e-09'